In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer

In [4]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    df = pd.read_csv(file_path)
    return df

In [5]:
def prepare_data_part1(df, test_size=0.2, random_state=42):
    """
    Prepare data for modeling: select features, split into train/test sets, handle missing values.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    
    # 1. Select relevant features (age, systolic_bp, diastolic_bp, glucose_level, bmi)
    features = ['age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi']
    X = df[features]
    
    # 2. Select target variable (disease_outcome)
    y = df['disease_outcome']
    
    # 3. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # 4. Handle missing values using SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [6]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    
    # Initialize and train a LogisticRegression model
    model = LogisticRegression()
    model.fit(X_train,y_train)
    
    return model


In [7]:
def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    
    # 1. Generate predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    # 3. Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # 4. Return metrics in a dictionary
    return {
        'accuracy' : acc,
        'precision' : prec,
        'recall' : rec,
        'f1_score' : f1,
        'auc' : auc,
        'confusion_matrix' : cm
    }

In [8]:
def save_results(metrics):
    # 6. Save results
    results_dir = 'results'
    os.makedirs(results_dir, exist_ok=True)

    results_path = os.path.join(results_dir, 'results_part1.txt')
    with open(results_path, 'w') as f:
        for metric, value in metrics.items():
            if metric == 'confusion_matrix':
                f.write("confusion_matrix:\n")
                f.write(str(value) + "\n")
            else:
                f.write(f"{metric}: {value:.4f}\n")

In [9]:
def interpret_results(metrics):
    """
    Analyze model performance on imbalanced data.
    
    Args:
        metrics: Dictionary containing evaluation metrics
        
    Returns:
        Dictionary with keys:
        - 'best_metric': Name of the metric that performed best
        - 'worst_metric': Name of the metric that performed worst
        - 'imbalance_impact_score': A score from 0-1 indicating how much
          the class imbalance affected results (0=no impact, 1=severe impact)
    """

    scalar_metrics = {k: v for k, v in metrics.items() if k != 'confusion_matrix'}
    # 1. Determine which metric performed best and worst
    best_metric = max(scalar_metrics, key=scalar_metrics.get)
    worst_metric = min(scalar_metrics, key=scalar_metrics.get)
    
    # 2. Calculate an imbalance impact score based on the difference
    #    between accuracy and more imbalance-sensitive metrics like F1 or recall
    accuracy = scalar_metrics.get('accuracy', 0)
    f1 = scalar_metrics.get('f1_score', 0)
    recall = scalar_metrics.get('recall', 0)
    
    imbalance_impact_score = abs(accuracy - ((f1 + recall) / 2))
    imbalance_impact_score = round(min(1.0, imbalance_impact_score), 4)
    
    # 3. Return the results as a dictionary
    return {
        'best_metric': best_metric,
        'worst_metric': worst_metric,
        'imbalance_impact_score': imbalance_impact_score
    }

In [11]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data
    X_train, X_test, y_train, y_test = prepare_data_part1(df)
    
    # 3. Train model
    model = train_logistic_regression(X_train, y_train)
    
    # 4. Evaluate model
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 5. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 6. Save results
    save_results(metrics=metrics)

    # 7. Interpret results
    interpretation = interpret_results(metrics)
    print("\nResults Interpretation:")
    for key, value in interpretation.items():
        print(f"{key}: {value}")

accuracy: 0.9168
precision: 0.6615
recall: 0.3007
f1_score: 0.4135
auc: 0.9084

Results Interpretation:
best_metric: accuracy
worst_metric: recall
imbalance_impact_score: 0.5597
